# 映画の評価データセット

データの取得
- https://cran.r-project.org/web/packages/recommenderlab/readme/README.html

データの説明
- https://www.rdocumentation.org/packages/recommenderlab/versions/0.2-6/topics/MovieLense
- MovieLens (umn.edu) により収集された映画の評価
- 映画の数：1664, ユーザ数：943

### CSVファイルフォーマット

CSV (Comma-Separated Values)
- フィールド（列）をカンマ , で区切ったデータ形式
- 規格 RFC 4180

例：
```
123,456,789
```

### CSVファイルの読み込み

最低限の注意点：
- フィールド（列）の区切り文字
- 欠損値を表す文字列

In [ ]:
import numpy as np
import pandas as pd

# read_csv によりCSVファイルを読み込み
# - na_values により欠損値の文字列 'NA' を指定
movie = pd.read_csv('data/MovieLense.csv', na_values='NA')

# 確認
movie.head()

In [ ]:
# 不要な先頭列 'Unnamed: 0' を削除
df = movie.drop('Unnamed: 0', axis=1)

# 確認
df.head()

### 列インデックスの数値化

列インデックスに記号（' など）が入っていると問題を起こす可能性があるため、インデックスは数値化する。
別途、インデックスからタイトルを引くためのDataFrameを作成する。

In [ ]:
# 列名をデータとするDataFrameを作成
df_titles = pd.DataFrame(df.columns, columns=['title'])
# 確認
df_titles.head(5)

In [ ]:
# 列インデックスの数値化
df.columns = df_titles.index
# 確認
df.head(1)

### 類似度を計算する関数の定義

- ユークリッド距離（1を足して値の逆数）
- 相関係数
- コサイン類似度

In [ ]:
# ユークリッド距離（1を足して値の逆数）
from scipy.spatial.distance import euclidean

def similarity_euclide(df, l1, l2):
    a = df.loc[[l1, l2]].dropna(axis=1)
    if len(a.iloc[0]) < 2:
        return np.nan
    return 1 / (1 + euclidean(a.iloc[0], a.iloc[1]))

# 相関係数
def similarity_correlation(df, l1, l2):
    _df = df.loc[[l1, l2]].dropna(axis=1).T
    return _df.corr().iloc[0, 1]

# コサイン類似度（= 1 - コサイン距離）
from scipy.spatial.distance import cosine

def similarity_cosine(df, l1, l2):
    a = df[[l1, l2]].dropna()
    if len(a[l1]) < 2:
        return np.nan
    return 1 - cosine(a[l1], a[l2])

### 評価の予測

- 入力：評価値データベース、予測したい評価があるユーザ
- 出力：予測値とアイテムのリスト

In [ ]:
def user_base_scores(db, user, similarity_func=similarity_correlation):
    # 初期値
    scores = []
    # 処理用にDataFrameを複製
    df = db.copy()
    # 平均評価値の計算
    mean = df.mean(axis=1)
    # 類似度列を追加し、NaN で初期化
    df['similarity'] = np.nan
    # 他ユーザについて類似度を計算し、DataFrameに代入
    for other in df.index.drop(user):
        df.at[other, 'similarity'] = similarity_func(df, other, user)
    # 平均評価値列の追加
    df['mean'] = mean
    # 正の相関を持つユーザのみ（ユークリッド距離の場合は全て正）
    df_sim = df.query('similarity > 0')
    # user 行で　NaN が入っている列について処理
    for i in db.columns[db.loc[user].isnull()]:
        # 加重平均を計算
        avg = ((df_sim[i] - df_sim['mean']) * df_sim.similarity).sum()
        avg = avg / df_sim.similarity.sum()
        # 平均評価値に加算
        scores.append([i, df.at[user, 'mean'] + avg])
    # 予測値を返す
    return scores

def item_base_scores(db, user, item):
    # user 行 item 列
    if not np.isnan(db.loc[user, item]):
        return [item, db.loc[user, item]]

    # 初期値
    scores = []
    # 処理用にDataFrameを複製
    df = db.copy()
    # 平均評価値の計算
    mean = df.mean(axis=1)
    # 列の平均値を計算し、平均評価値の列を追加
    df['mean'] = mean
    # 平均評価値を引き調整
    for i in df.columns.drop('mean'):
        df[i] = df[i] - df['mean']
    # mean 列を削除
    df = df.drop('mean', axis=1)
    # 類似度行を追加し、NaN で初期化
    df.loc['similarity'] = np.nan

    for i in df.columns.drop(item):
        df.at['similarity', i] = similarity_cosine(df, i, item)
    # 正の類似度を持つアイテムのみを取り出す
    # - query は行に対してのみ処理可能なため T で転置する
    df_sim = df.T.query('similarity > 0')
    # 加重平均を計算
    avg = (df_sim[user] * df_sim.similarity).sum() / df_sim.similarity.sum()

    # 平均評価値に加算した予測値を返す
    return [item, mean[user] + avg]

### ユーザベース

In [ ]:
# ユーザベースの予測値を計算し、DataFrameを作成
rec = pd.DataFrame(user_base_scores(df, 0), columns=['tid', 'score'])
# tid（タイトルのインデックス）を行インデックスとして設定
# - タイトルのインデックスと対応付けないと、次のタイトルのDataFrameとの結合がうまくいかない
rec = rec.set_index('tid')
# 確認
rec

In [ ]:
# タイトル列の追加
rec['title'] = df_titles.loc[rec.index]
# 確認
rec

In [ ]:
# 予測値上位5つの取得
rec_top5 = rec.sort_values('score', ascending=False).head(5)
# 確認
rec_top5

In [ ]:
# ユークリッド類似度で計算
rec_euclide = pd.DataFrame(user_base_scores(df, 0, similarity_func=similarity_euclide),
                           columns=['tid', 'score']).set_index('tid')
rec_euclide['title'] = df_titles.loc[rec_euclide.index]
rec_euclide_top5 = rec_euclide.sort_values('score', ascending=False).head(5)
# 確認
rec_euclide_top5

### アイテムベース

- 予測値をアイテムごとに計算する方法

In [ ]:
item_base_scores(df, 0, 310)

In [ ]:
item_base_scores(df, 0, 300)

In [ ]:
item_base_scores(df, 0, 315)

### アイテムベース (2)

- 類似度を事前に計算する方法
- 類似度を計算には時間がかかるので各自で実行

In [ ]:
# 類似度をアイテムごとに計算
def calc_item_base_similarity(db):
    # 処理用にDataFrameを複製
    df = db.copy()
    # 列の平均値を計算し、平均評価値の列を追加
    df['mean'] = df.mean(axis=1)
    # 平均評価値を引き調整
    for i in df.columns.drop('mean'):
        df[i] = df[i] - df['mean']
    # mean 列を削除
    mean_db = df.drop('mean', axis=1)
    # 類似度表の初期化
    sim_db = pd.DataFrame(index=db.columns, columns=db.columns)
    # 類似度表の計算
    for i in mean_db.columns:
        for j in mean_db.columns.drop(i):
            # 既に埋まっていなければ計算
            if np.isnan(sim_db.at[i, j]):
                sim_db.at[i, j] = similarity_cosine(mean_db, i, j)
                # 対象成分にも代入
                sim_db.at[j, i] = sim_db.at[i, j]
    return mean_db, sim_db

def item_base_scores(db, item_base_db, user):
    # 初期値
    scores = []
    # 平均評価値の計算
    mean = db.loc[user].mean()
    # 平均評価値および類似度を取得
    mean_db = item_base_db[0]
    sim_db = item_base_db[1]
    # user 行で　NaN が入っている列について処理
    for row in db.columns[db.loc[user].isnull()]:
        # 正の類似度のみを取り出す
        #df= sim_db[[row]].query('{} > 0'.format(row))
        _sim_db = sim_db[[row]]
        df = _sim_db[_sim_db > 0]
        # 正の類似度に対応する平均評価値を取り出す
        df['mean'] = mean_db.loc[user, df.index]
        # 加重平均を計算
        # - 評価値がない列の平均は 0
        row_sum = df[row].sum()
        if (row_sum != 0):
            avg = (df[row] * df['mean']).sum() / row_sum
        else:
            avg = 0
        # 平均評価値に加算
        scores.append([row, mean + avg])

    # 予測値を返す
    return scores

In [ ]:
%%time
# Mac mini Core i7 3.2GHz：約40分
# MacBook Pro M1 Pro：約15分
item_base_db = calc_item_base_similarity(df)

In [ ]:
rec = pd.DataFrame(item_base_scores(df, item_base_db, 0), columns=['tid', 'score']).set_index('tid')
rec['title'] = df_titles.loc[rec.index]
rec_top5 = rec.sort_values('score', ascending=False).head(5)
# 確認
rec_top5